In [14]:
import cv2,os,random,numpy as np

In [2]:
#getting emoji name list
def getEmojis():
    emojis_folder = 'emojis/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

In [3]:
def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image    
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))


#resizing the the emoji 
def blend(image, emoji, position):
    x, y, w, h = position
    emoji = cv2.resize(emoji, (w, h))
    try:
    image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image


In [12]:
def emoFace():
    
    emojis = getEmojis()
    
    faceDetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    cam=cv2.VideoCapture(0)
    
    r=6
    

    while(True):
        
        rett,img=cam.read()
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces=faceDetect.detectMultiScale(gray,1.3,5)
        if (cv2.waitKey(1)==ord("w")):
            r=random.randint(0,len(emojis)-1)
        for(x,y,w,h) in faces:

            #cv.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
            """x,y=int(x*0.7),int(y*0.7)
            w,h=int(w*2),int(h*1.7)"""
            img = blend(img, emojis[r], (x, y, w, h))

        cv2.imshow("Face",img)
        
        if (cv2.waitKey(1)==ord("q")):
            break
    cam.release()
    cv2.destroyWindow("Face")

In [13]:
emoFace()
